In [19]:
from IPython.display import display
from save_load_csv import load_csv
import ipyleaflet as leaflet
import ipywidgets as widgets

# Static variables for parsing
neighbourhood_group_list = ['Manhattan', 'Brooklyn', 'Bronx', 'Queens', 'Staten Island']
room_type_list = ['Entire home/apt', 'Private room', 'Shared room']
desired_keys = ['neighbourhood_group', 'neighbourhood', 'latitude', 'longitude', 'price', 'room_type', 'minimum_nights']
data = load_csv('./AB_NYC_2019.csv')
print(len(data))

# Filter data with desired keys
filtered_data = []
for sample in data:
    filtered_data.append({key: sample[key] for key in desired_keys})

# Cleanup used data
del data

# Initialize map and layer object
m = leaflet.Map(center=(0,0), zoom=1)
m.add_control(leaflet.LayersControl())
layer_group = leaflet.LayerGroup()

# Filter dictionary
filter_value = {'neighbourhood_group': '',
                'neighbourhood': '',
                'room_type': '',
                'price_min': 0,
                'price_max': 100,
                'nights': 1
               }

# Filter function
def predicate(sample):
    if (sample['neighbourhood_group']==filter_value['neighbourhood_group'] and
        sample['neighbourhood']==filter_value['neighbourhood'] and
        sample['room_type']==filter_value['room_type'] and
        int(sample['price']) <= filter_value['price_max'] and
        int(sample['minimum_nights']) <= filter_value['nights']):
        return True
    else:
        return False

# Filtering
def filter_input(data): return list(filter(predicate, data))

# Update Map
def update_map(neighbourhood_group,
               neighbourhood,
               room_type,
               price_max,
               nights):
    # Update filter value
    filter_value['neighbourhood_group'] = neighbourhood_group
    filter_value['neighbourhood'] = neighbourhood
    filter_value['room_type'] = room_type
    filter_value['price_max'] = price_max
    filter_value['nights'] = nights
    
    # Filter data by input param
    output = filter_input(filtered_data)
    if len(output)==0:
        print('Warning: no listings, which can be caused by wrong filter values')
        return
    print('Numer of listings: {}'.format(len(output)))
    
    # Get center and zoom
    # Get coordinates from filter output (lat, long)
    coordinates = []
    lat_center, lng_center = 0, 0
    min_lat, max_lat, min_lng, max_lng = 0, 0, 0, 0
    for sample in output:
        coordinates.append((float(sample['latitude']), float(sample['longitude'])))
        lat_center += float(sample['latitude'])
        lng_center += float(sample['longitude'])
    lat_center /= len(output)
    lng_center /= len(output)

    # Get min and max lat for zoom calculation
    lat_min_max = min(coordinates)[0], max(coordinates)[0]
    lat_range = lat_min_max[1]-lat_min_max[0]
    lng_min_max = min(coordinates)[1], max(coordinates)[1]
    lng_range = lng_min_max[1]-lng_min_max[0]
    if lat_range > lng_range:
        max_range = lat_range
    else:
        max_range = lng_range

    # Generate center and zoom for map
    center = (lat_center, lng_center) # geological center of US
    zoom = int(np.floor(-3.3333*max_range + 13.6667))
    if zoom > 18:
        zoom = 18
    elif zoom < 1:
        zoom = 1
        
    # Reset map
    if len(m.layers) > 1:
        m.remove_layer(m.layers[1])
    m.center = center
    m.zoom = zoom

    # Clean the circles layer group from previous trigger
    if len(layer_group.layers) != 0:
        for layer in layer_group.layers:
            layer_group.remove_layer(layer)

    # Add new circles to layer group
    for i in range(len(coordinates)):
        layer_group.add_layer(leaflet.Circle(location=coordinates[i], radius=3, color='red', fill_color='red'))

    # Add layer group to map
    m.add_layer(layer_group)
    
# Create widgets for map
neighbourhood_group_widget = widgets.Dropdown(
    options=neighbourhood_group_list,
    #value='',
    description='Group:',
    disabled=False,
)

neighbourhood_widget = widgets.Text(
    #value='',
    placeholder='e.g. Harlem',
    description='Area:',
    disabled=False
)
            
room_type_widget = widgets.Dropdown(
    options=room_type_list,
    #value='',
    description='Room Type:',
    disabled=False,
)

max_price_widget = widgets.BoundedIntText(
    value=1,
    min=0,
    max=10000,
    step=1,
    description='Max Price:',
    disabled=False
)
            
nights_widget = widgets.BoundedIntText(
    value=1,
    min=1,
    max=1250,
    step=1,
    description='Nights:',
    disabled=False
)

button_widget = widgets.Button(description="Update")
def on_button_clicked(b):
    update_map(neighbourhood_group_widget.value,
               neighbourhood_widget.value,
               room_type_widget.value,
               max_price_widget.value,
               nights_widget.value
              )
button_widget.on_click(on_button_clicked)

# Display widgets
display(neighbourhood_group_widget)
display(neighbourhood_widget)
display(room_type_widget)
display(max_price_widget)
display(nights_widget)
display(button_widget)

# Display map
m

Dropdown(description='Group:', options=('Manhattan', 'Brooklyn', 'Bronx', 'Queens', 'Staten Island'), value='M…

Text(value='', description='Area:', placeholder='e.g. Harlem')

Dropdown(description='Room Type:', options=('Entire home/apt', 'Private room', 'Shared room'), value='Entire h…

BoundedIntText(value=1, description='Max Price:', max=10000)

BoundedIntText(value=1, description='Nights:', max=1250, min=1)

Button(description='Update', style=ButtonStyle())

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …